In [2]:
import transformers
from datasets import load_dataset, load_metric

In [3]:
wikisql_processed_dataset = load_dataset("csv", data_files="datasets/eng2SQL_raw.csv")

wikisql_processed_dataset

Found cached dataset csv (/home/daniil/.cache/huggingface/datasets/csv/default-5d3942e87f7691cd/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 56355
    })
})

In [4]:
datasets_train_test = wikisql_processed_dataset["train"].train_test_split(test_size=2000)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=2000)

wikisql_processed_dataset["train"] = datasets_train_validation["train"]
wikisql_processed_dataset["validation"] = datasets_train_validation["test"]
wikisql_processed_dataset["test"] = datasets_train_test["test"]

wikisql_processed_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 52355
    })
    validation: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 2000
    })
})

In [5]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=512)

[nltk_data] Downloading package punkt to /home/daniil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# keep only a subsample of the datasets
wikisql_processed_dataset["train"] = wikisql_processed_dataset["train"].shuffle().select(range(50000))
wikisql_processed_dataset["validation"] = wikisql_processed_dataset["validation"].shuffle().select(range(1000))
wikisql_processed_dataset["test"] = wikisql_processed_dataset["test"].shuffle().select(range(1000))

wikisql_processed_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'human_sql'],
        num_rows: 1000
    })
})

In [7]:
prefix = "translate English to SQL: "
max_input_length = 512
max_target_length = 128

def preprocess_data(examples):
    inputs = [prefix + text for text in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["human_sql"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_datasets = wikisql_processed_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/home/daniil/torch/graduation-work/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [10]:
batch_size = 8
model_name = "t5-small-english-to-sql-raw-translation"
model_dir = f"models/{model_name}"

print(transformers.__version__)

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    report_to="tensorboard"
)

4.28.0


In [11]:
import numpy as np

data_collator = DataCollatorForSeq2Seq(tokenizer)

metric = load_metric("bleu")

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    predictions = [pred.split() for pred in predictions]
    
    # Replace -100 in the labels as we can't decode them.
    references = np.where(references != -100, references, tokenizer.pad_token_id)
    references = tokenizer.batch_decode(references, skip_special_tokens=True)
    references = [ref.split() for ref in references]
    references = [[ref] for ref in references]
    
    
    # Compute BLEU scores
    result = metric.compute(predictions=predictions, references=references)

    # Extract ROUGE f1 scores
    # result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)
    
    return result

/tmp/ipykernel_5852/909098219.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("bleu")


In [12]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [12]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

In [13]:
trainer.train()

/home/daniil/torch/graduation-work/venv/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length
100,3.279200,1.600417,0.181706,"[0.5535853685736589, 0.3063888540228442, 0.19865078225922206, 0.10631895687061184]",0.742729,0.770758,8967,11634
200,1.674600,1.236402,0.346293,"[0.6930783242258652, 0.47613150191398335, 0.35705225773718924, 0.24803950043566658]",0.837535,0.849407,9882,11634
300,1.367100,1.067369,0.390736,"[0.7549382716049383, 0.5360091743119266, 0.4134403729120808, 0.3062741599762117]",0.821262,0.835482,9720,11634
400,1.208700,0.967295,0.421017,"[0.7730445493738453, 0.5666895447266072, 0.44894797986317286, 0.3474011550422035]",0.823484,0.837373,9742,11634
500,1.069500,0.897922,0.444973,"[0.7899117020196894, 0.5882751609623856, 0.47021384928716903, 0.36973319725907566]",0.834638,0.846914,9853,11634
600,0.988800,0.844253,0.465431,"[0.7991504854368932, 0.6073357335733574, 0.49359868170870835, 0.3969521044992743]",0.838134,0.849923,9888,11634
700,0.949100,0.807839,0.479121,"[0.8077895801719778, 0.6227349465391109, 0.5101445599797109, 0.4167271671264702]",0.837834,0.849665,9885,11634
800,0.951000,0.769996,0.489040,"[0.8133400707427993, 0.6345137717818999, 0.5213426219126029, 0.4293793503480278]",0.838832,0.850524,9895,11634
900,0.897000,0.749200,0.493899,"[0.81591737545565, 0.6411671924290221, 0.5298374809547994, 0.4375454413261597]",0.836936,0.848891,9876,11634
1000,0.900600,0.726063,0.499194,"[0.8192136197811106, 0.6472710870545783, 0.5374936451448907, 0.44577085456398313]",0.836137,0.848204,9868,11634


Trainer is attempting to log a value of "[0.5535853685736589, 0.3063888540228442, 0.19865078225922206, 0.10631895687061184]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.6930783242258652, 0.47613150191398335, 0.35705225773718924, 0.24803950043566658]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.7549382716049383, 0.5360091743119266, 0.4134403729120808, 0.3062741599762117]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.7730445493738453, 0.5666895447266072, 0.44894797986317286, 0.3474011550422035]" of type <class 'list'> for k

Trainer is attempting to log a value of "[0.8436498397435898, 0.70013357079252, 0.6019539078156313, 0.5216208476517755]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8456907795725895, 0.7035797925727668, 0.605623195682189, 0.5261949189034018]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8433530059017705, 0.7025675225075025, 0.607102663498812, 0.5279405459482636]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8445310937812438, 0.7027327260608753, 0.6069732566858286, 0.5274207369323051]" of type <class 'list'> for key "eval/p

Trainer is attempting to log a value of "[0.860413737698333, 0.7324179504353651, 0.6433777330987686, 0.5695602184535786]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8606285771663822, 0.7335640138408305, 0.6444276919210956, 0.5706279637879006]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8587032768814511, 0.7322641719567881, 0.6438150144128337, 0.569995701389884]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8583016222711797, 0.7300244825283775, 0.6409967443025294, 0.5665616948182078]" of type <class 'list'> for key "eval

Trainer is attempting to log a value of "[0.8650140505820956, 0.7396251673360107, 0.6515570065293822, 0.5786904078116025]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8656716417910447, 0.739953245018368, 0.6516347237880497, 0.5785479020478305]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.863431715857929, 0.7352973874374653, 0.6469043151969981, 0.5736954967834167]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8652787508757882, 0.7388499610721833, 0.650857214366162, 0.5777428121870977]" of type <class 'list'> for key "eval/

Trainer is attempting to log a value of "[0.8711, 0.7501111111111111, 0.664, 0.593]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8700129987001299, 0.7470281079880013, 0.6606674165729284, 0.589487216111984]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8695086560592414, 0.7471366618481041, 0.6610784436381834, 0.5901615901615902]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8704870487048705, 0.7479719968885432, 0.6618327290911364, 0.5902271753107586]" of type <class 'list'> for key "eval/precisions" as a scalar. This invoca

Trainer is attempting to log a value of "[0.872336168084042, 0.7520844913841023, 0.6675422138836773, 0.5967119370979271]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8724127587241276, 0.7532496389290079, 0.6687914010748657, 0.5982002571061277]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8724852367130418, 0.7517517517517518, 0.6663746715054436, 0.5949077385209555]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8712970376301041, 0.7512233096085409, 0.6665415415415415, 0.5952517162471396]" of type <class 'list'> for key "eva

Trainer is attempting to log a value of "[0.871864067966017, 0.751471404775125, 0.6668332292317302, 0.5965738758029978]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8719896072749076, 0.7515265904296659, 0.6667915573872861, 0.5964035964035964]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8717897471769761, 0.7510824913955813, 0.6662919945048082, 0.5958327386898815]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8717769338396962, 0.7510548523206751, 0.6665001249063203, 0.5960605195546674]" of type <class 'list'> for key "eval

TrainOutput(global_step=18750, training_loss=0.594239241027832, metrics={'train_runtime': 4587.4301, 'train_samples_per_second': 32.698, 'train_steps_per_second': 4.087, 'total_flos': 1357628413575168.0, 'train_loss': 0.594239241027832, 'epoch': 3.0})

In [13]:
model_name = "t5-small-english-to-sql-raw-translation/checkpoint-18600"
model_dir = f"models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [14]:
text = "What circuit was the FR3.5 11 series on?"

inputs = ["translate English to SQL: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=4, max_length=128)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_query = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_query) # SELECT Circuit FROM table WHERE Series = FR3.5 11

SELECT Circuit FROM table WHERE Series = FR3.5 11


In [24]:
text = "How many numbers of languages were there for issues with circulation of 10,748,000?"

inputs = ["translate English to SQL: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=4, max_length=128)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_query = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_query) # SELECT MAX Number of languages FROM table WHERE Circulation per issue = 10,748,000

SELECT COUNT Language FROM table WHERE Circumstance = 10,748,000


In [23]:
text = "Which team has the greatest number of points?"

inputs = ["translate English to SQL: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=4, max_length=128)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_query = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_query)

SELECT MAX Points FROM table


In [22]:
text = "Who was the opposing team at Schaefer Stadium later in the season than week 7?   "

inputs = ["translate English to SQL: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=4, max_length=128)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_query = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_query) # SELECT Opponent FROM table WHERE Week > 7 AND Stadium = schaefer stadium

SELECT Opponent FROM table WHERE Week > 7 AND Venue = schaefer stadium


In [19]:
text = "Who has won Australian Grand Prix?       "

inputs = ["translate English to SQL: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=4, max_length=128)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_query = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_query) # SELECT Winning Driver FROM table WHERE Grand Prix = Australian Grand Prix

SELECT Winner FROM table WHERE Grand Prix = australian grand prix
